In [1]:
from IPython.display import Markdown, Latex
from datasets import load_dataset
import numpy as np
import pandas as pd

In [2]:
methods = [
    # "danjacobellis/MUSDB_opus_12kbps",
    "danjacobellis/MUSDB_Stereo_Li_108c_test2",
    "danjacobellis/MUSDB_encodec_12kbps",
    "danjacobellis/MUSDB_Stereo_Li_27c_test2",
    "danjacobellis/MUSDB_stable_audio_fp16"
]

dimensionality_reduction = [
    # 1,
    512/108,
    5,
    512/27,
    64,
]

names = [
    # 'Opus',
    'WalloC',
    'EnCodec',
    'WalloC',
    'Stable Audio',
]

metrics = [
    'PSNR',
    'SSDR',
    'SRDR',
]

columns = [
    'Name',
    'Dimensionality Reduction',
    'Compression Ratio',
    'Encode (CPU)',
    'Decode (CPU)',
    'Encode (GPU)',
    'Decode (GPU)',
] + metrics + ['CDPAM']

In [3]:
data_rows = []

for i_method, method in enumerate(methods):
    ds = load_dataset(method, split='validation')
    row = [names[i_method], dimensionality_reduction[i_method]]
    bps_mean = np.mean(ds['bps'])
    compression_ratio = 16 / bps_mean
    encode_MS_per_second = 2*np.mean(np.array(ds['L'])/np.array(ds['cpu_encode_time'])/1e6)
    decode_MS_per_second = 2*np.mean(np.array(ds['L'])/np.array(ds['cpu_decode_time'])/1e6)
    encode_MS_per_second_GPU = 2*np.mean(np.array(ds['L'])/np.array(ds['encode_time'])/1e6)
    decode_MS_per_second_GPU = 2*np.mean(np.array(ds['L'])/np.array(ds['decode_time'])/1e6)
    row.append(compression_ratio)
    row.append(encode_MS_per_second)
    row.append(decode_MS_per_second)
    row.append(encode_MS_per_second_GPU)
    row.append(decode_MS_per_second_GPU)
    for metric in metrics:
        mean = np.mean(ds[metric])
        row.append(mean)
    CDPAM_dB = np.median(-10*np.log10(ds['CDPAM']))
    row.append(CDPAM_dB)
    data_rows.append(row)

df = pd.DataFrame(data_rows, columns=columns)

df_rounded = df.round(4)
md_table = df_rounded.to_markdown(index=False)
display(Markdown(md_table))

Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

| Name         |   Dimensionality Reduction |   Compression Ratio |   Encode (CPU) |   Decode (CPU) |   Encode (GPU) |   Decode (GPU) |    PSNR |    SSDR |    SRDR |   CDPAM |
|:-------------|---------------------------:|--------------------:|---------------:|---------------:|---------------:|---------------:|--------:|--------:|--------:|--------:|
| WalloC       |                     4.7407 |             21.3065 |        77.7947 |        11.1713 |       161.422  |       654.311  | 39.0242 | 33.2625 | 13.8961 | 41.0717 |
| EnCodec      |                     5      |            114.131  |         2.7511 |         3.0337 |        10.6584 |        12.4735 | 31.9516 | 22.6949 |  6.6919 | 47.3817 |
| WalloC       |                    18.963  |             76.2823 |       121.26   |        12.1872 |       385.775  |      1055.48   | 33.3161 | 22.523  |  8.061  | 36.641  |
| Stable Audio |                    64      |             64      |         0.3084 |         0.2998 |        24.7086 |      1667.82   | 28.4155 | 15.7432 |  2.0318 | 49.7486 |